In [14]:
import pandas as pd
pd.set_option('display.max_columns', None)

import sys
sys.path.append(r'C:\Users\huzef\OneDrive\Documents\Projects\Projects\data_science_code')

from data_processor import DataProcessor
from classification_models import LogisticRegressionModel, RandomForestModel
from convert_categorical_variables import CategoricalEncoder
from evaluate_classification import ClassificationEvaluator

ImportError: cannot import name 'RandomForestModel' from 'classification_models' (C:\Users\huzef\OneDrive\Documents\Projects\Projects\data_science_code\classification_models.py)

In [2]:
data_processor = DataProcessor()
train_df = data_processor.read_csv(r'C:\Users\huzef\OneDrive\Documents\Projects\Projects\data\titanic\train.csv')
test_df = data_processor.read_csv(r'C:\Users\huzef\OneDrive\Documents\Projects\Projects\data\titanic\test.csv')
gender_submission = data_processor.read_csv(r'C:\Users\huzef\OneDrive\Documents\Projects\Projects\data\titanic\gender_submission.csv')

In [3]:
train_data, test_data = data_processor.train_test_split(train_df, test_size=0.2, random_state=42)
X_train, y_train = data_processor.split_features_target(train_data, 'Survived')
X_test, y_test = data_processor.split_features_target(test_data, 'Survived')

Data split into training and testing sets successfully.
Features and target variable split successfully.
Features and target variable split successfully.


# EDA/ Feature Engineering

In [4]:
def eda(df_1):
    df = df_1.copy()
    df.drop('PassengerId', axis=1, inplace=True)

    df['Surname'] = df['Name'].str.split(',').str[0]
    df['FirstName'] = df['Name'].str.split(',').str[1]
    df['Title'] = df['FirstName'].str.split('.').str[0]
    df.drop(['Name', 'FirstName', 'Surname'], axis=1, inplace=True)
    rare_titles = ['Rev', 'Dr', 'Major', 'Col', 'Mlle', 'Capt', 'Mme', 'the Countess', 'Lady', 'Sir', 'Jonkheer', 'Don']
    df['Title'] = df['Title'].apply(lambda x: 'Miss' if x.strip() == 'Ms' else 'Rare' if x.strip() in rare_titles else x.strip())

    df['Sex'] = df['Sex'].map({'male': 1, 'female': 0})

    df['Ticket_string'] = df['Ticket'].str.contains('[a-zA-Z]').astype(int)

    df['Cabin'] = df['Cabin'].fillna('U')

    most_common = df['Embarked'].mode()[0]
    df['Embarked'] = df['Embarked'].fillna(most_common)

    df['Age'] = df['Age'].fillna(df['Age'].median())

    df['Cabin_Letter'] = df['Cabin'].str.extract(r'([A-Za-z])')
    df['Cabin_Number'] = df['Cabin'].str.extract(r'(\d+)')
    df['Cabin_Number'] = df['Cabin_Number'].fillna(0)
    
    df = df[['Pclass','Sex','Age','SibSp','Parch','Fare','Embarked','Title','Ticket_string','Cabin_Letter','Cabin_Number']]

    encoder = CategoricalEncoder(df)
    df = encoder.one_hot_encoding(['Embarked', 'Title', 'Cabin_Letter'])
    df = df.drop(['Cabin_Letter_U'], axis=1)

    return df

X_train_filt = eda(X_train)
X_test_filt = eda(X_test)

X_train_filt = X_train_filt[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Ticket_string',
       'Cabin_Number', 'Embarked_C', 'Embarked_Q', 'Embarked_S',
       'Title_Master', 'Title_Miss', 'Title_Mr', 'Title_Mrs', 'Title_Rare',
       'Cabin_Letter_A', 'Cabin_Letter_B', 'Cabin_Letter_C', 'Cabin_Letter_D',
       'Cabin_Letter_E', 'Cabin_Letter_F', 'Cabin_Letter_G']]

In [11]:
log_reg_model = LogisticRegressionModel()
log_reg_model.set_params(random_state=42, max_iter=1000) 
log_reg_model.train(X_train_filt, y_train)
log_reg_predict = log_reg_model.predict(X_test_filt)

In [13]:
rf_model = RandomForestModel(random_state=42)
rf_model.train(X_train_filt, y_train)
predictions1 = rf_model.predict(X_test_filt)
best_params = rf_model.tune_hyperparameters(X_train_filt, y_train, param_grid={'n_estimators': [100, 200], 'max_depth': [10, 20]})
predictions2 = rf_model.predict(X_test_filt)
rf_model.train(X_train_filt, y_train)
predictions3 = rf_model.predict(X_test_filt)

NameError: name 'RandomForestModel' is not defined

In [12]:
evaluator = ClassificationEvaluator(y_true=y_test)
evaluator.add_model('Logistic Regression', log_reg_predict)
evaluator.evaluate_all_models()

,Model,Accuracy,Precision,Recall,F1 Score,Confusion Matrix,ROC AUC
0,Logistic Regression,0.821229,0.776316,0.797297,0.786667,88\t17\n15\t59,None
